<a href="https://colab.research.google.com/github/b62mp1/files/blob/main/Final_C1_Dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install yfinance

In [17]:
pip install -U kaleido

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go
from google.colab import files

In [19]:
stock_name = "TSLA"
dataset=pdr.get_data_yahoo(stock_name, start='2020-01-01', end='2022-04-19').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (578, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-01-02,2020-01-02,84.900002,86.139999,84.342003,86.052002,86.052002,47660500
2020-01-03,2020-01-03,88.099998,90.800003,87.384003,88.601997,88.601997,88892500
2020-01-06,2020-01-06,88.094002,90.311996,88.000000,90.307999,90.307999,50665000
2020-01-07,2020-01-07,92.279999,94.325996,90.671997,93.811996,93.811996,89410500
2020-01-08,2020-01-08,94.739998,99.697998,93.646004,98.428001,98.428001,155721500


In [20]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)

In [21]:

def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates


In [22]:
ls = [0.20, 0.80]
for i in ls:
  split_percent = i
  split = int(split_percent*len(close_data))

  close_train = close_data[:split]
  close_test = close_data[split:]

  date_train = dataset['Date'][:split]
  date_test = dataset['Date'][split:]

  look_back = 5

  train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
  test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=10)

  model = Sequential()
  model.add(
      LSTM(1000,
          activation='relu',
          input_shape=(look_back,1)
          )
  )

  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')

  num_epochs = 25
  model.fit(train_generator,epochs=num_epochs, verbose=1)

  prediction = model.predict(test_generator)

  num_prediction = 30
  forecast = predict(num_prediction, model)
  forecast_dates = predict_dates(num_prediction)

  #close_test=close_test[:-look_back]

  close_train = close_train.reshape((-1))
  close_test = close_test.reshape((-1))
  prediction = prediction.reshape((-1))

  trace1 = go.Scatter(
      x = date_train,
      y = close_train,
      mode = 'lines',
      name = 'Data',
  )


  trace2 = go.Scatter(
      x = date_test,
      y = prediction,
      mode = 'lines',
      name = 'Predicted Data'
  )
  trace3 = go.Scatter(
      x = date_test,
      y = close_test,
      mode='lines',
      name = 'Test Data'
  )
  trace4 = go.Scatter(
      x = forecast_dates,
      y = forecast,
      mode='lines',
      name = 'Future Data'
  )
  layout = go.Layout(
      title = stock_name,
      xaxis = {'title' : "Date"},
      yaxis = {'title' : "Close"}
  )
  fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
  #fig.update_xaxes(type="log")

  fig.write_image(str(i) + ".png")

  files.download(str(i) + ".png")

  close_test2=close_test[:-5]
  RNNmse = mean_squared_error(close_test2,prediction)
  from math import sqrt
  RNNrmse = sqrt(RNNmse)
  RNNr2Score = r2_score(close_test2,prediction)


  with open('Dates.txt', 'a') as testwritefile:
      testwritefile.write("Split % " + str(i) + " " + str(RNNmse) + " " + str(RNNrmse) + " " + str(RNNr2Score) + "\n")
files.download('Dates.txt')

Epoch 1/25
6/6 [==============================] - 3s 211ms/step - loss: 0.0017
Epoch 2/25
6/6 [==============================] - 1s 219ms/step - loss: 9.7825e-04
Epoch 3/25
6/6 [==============================] - 2s 317ms/step - loss: 4.4203e-04
Epoch 4/25
6/6 [==============================] - 2s 368ms/step - loss: 3.7005e-04
Epoch 5/25
6/6 [==============================] - 2s 274ms/step - loss: 4.7169e-04
Epoch 6/25
6/6 [==============================] - 2s 323ms/step - loss: 4.6424e-04
Epoch 7/25
6/6 [==============================] - 2s 301ms/step - loss: 5.7032e-04
Epoch 8/25
6/6 [==============================] - 2s 337ms/step - loss: 5.8596e-04
Epoch 9/25
6/6 [==============================] - 2s 355ms/step - loss: 2.7139e-04
Epoch 10/25
6/6 [==============================] - 2s 370ms/step - loss: 3.7089e-04
Epoch 11/25
6/6 [==============================] - 2s 385ms/step - loss: 2.2266e-04
Epoch 12/25
6/6 [==============================] - 2s 366ms/step - loss: 2.2665e-04
Epoch

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Epoch 1/25
23/23 [==============================] - 6s 189ms/step - loss: 0.0438
Epoch 2/25
23/23 [==============================] - 4s 190ms/step - loss: 0.0028
Epoch 3/25
23/23 [==============================] - 4s 190ms/step - loss: 0.0015
Epoch 4/25
23/23 [==============================] - 4s 175ms/step - loss: 0.0021
Epoch 5/25
23/23 [==============================] - 4s 177ms/step - loss: 0.0024
Epoch 6/25
23/23 [==============================] - 4s 182ms/step - loss: 0.0032
Epoch 7/25
23/23 [==============================] - 4s 177ms/step - loss: 0.0026
Epoch 8/25
23/23 [==============================] - 4s 178ms/step - loss: 0.0024
Epoch 9/25
23/23 [==============================] - 4s 176ms/step - loss: 0.0011
Epoch 10/25
23/23 [==============================] - 4s 174ms/step - loss: 0.0011
Epoch 11/25
23/23 [==============================] - 4s 178ms/step - loss: 9.7835e-04
Epoch 12/25
23/23 [==============================] - 4s 177ms/step - loss: 0.0021
Epoch 13/25
23/23 [==

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>